In [1]:
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
module_path = os.path.abspath(os.path.join('./src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [6]:
import pandas as pd
import numpy as np
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    N_REDUCED_SCNEARIOS,
    ALL_VEHICLE_TYPES,
    PATH_RESULTS_SUMMARY,
    PATH_RESULTS_SINGLE_MODAL_BENCHMARK,
    PATH_RESULTS_VALUE_STOCHASTIC,
    PATH_FLEET_SIZE,
    PATH_SCENARIO_TREE_NODES,
)
from modules.stochastic_program.factory import StochasticProgramFactory, StochasticProgram



# Stochastic Program
In this notebook we will use the previously prepared data to create a stochastic program and solve it. In order to evaluate its performance we will perform multiple benchmarks.
## Read Input Data

In [7]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [8]:
node_df = pd.read_pickle(PATH_SCENARIO_TREE_NODES)

In [9]:
distances = pd.read_pickle(PATH_DISTANCES)

In [10]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [11]:
real_fleet_size = pd.read_pickle(PATH_FLEET_SIZE).to_dict()["id"]
# TODO
real_fleet_size = {
    vehicle_type: int(fleet_size/3) for vehicle_type, fleet_size in real_fleet_size.items()
}
real_fleet_size

{'bicycle': 996, 'car': 658, 'kick_scooter': 2600}

# Create & Solve Stochastic Program

To create and solve the stochastic program we use the `StochasticProgramFactory` and `StochasticProgram` classes from our own module.  
We first create the stochastic program factory with all of the prepared data.

In [12]:
vehicle_types = ALL_VEHICLE_TYPES
factory = StochasticProgramFactory(
    scenarios,
    distances.apply(lambda x: round(x, 2)), # rounding improves performance
    probabilities,
    node_df,
    vehicle_types,
)
factory.set_initial_allocation(real_fleet_size)


_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.01 seconds
_convert_demand finished in 0.32 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.34 seconds
_set_max_demand finished in 0.06 seconds
set_initial_allocation finished in 0.01 seconds


Now we create the stochastic program using the factory. The factory will pass all the necessary data in the correct format to the stochastic program class.

In [13]:
stochastic_program: StochasticProgram = factory.create_stochastic_program()

create_stochastic_program finished in 0.00 seconds


We now use the stochastic program class to create the constraints and the objective function using the pulp library.  
We also can configure the model to be risk averse by setting a beta larger than zero. Beta is the weight of the calculate-value-at-risk. Look at the documentation in `stochastic_program.py` for further details.

In [14]:
stochastic_program.create_model(beta=0.5, alpha=0.75)

_create_variables finished in 2.09 seconds
_create_objectives finished in 4.15 seconds
_create_demand_constraints finished in 4.01 seconds
_create_relocation_binary_constraints finished in 0.03 seconds
_create_big_u_sum_constraints finished in 0.55 seconds
_create_unfulfilled_demand_binary_constraints finished in 0.04 seconds
_create_no_refused_demand_constraints finished in 0.03 seconds
_create_relocations_constraints finished in 1.52 seconds
_create_vehicle_trips_starting_constraints finished in 0.57 seconds
_create_vehicle_trips_ending_constraints finished in 0.62 seconds
_create_initial_allocation_constraints finished in 0.01 seconds
_create_non_anticipativity_constraints finished in 1.97 seconds
_create_value_at_risk_constraints finished in 2.86 seconds
_create_constraints finished in 12.22 seconds
create_model finished in 23.88 seconds


We now can solve the formulated program. Make sure that the solver selected in `config.py` is configured properly.

In [15]:
stochastic_program.solve()

No parameters matching '_test' found


Here we can take a look at the dimensions of the problem.

In [12]:
scenarios.reset_index().nunique()

scenarios          8
start_hex_ids     29
end_hex_ids       29
time               2
vehicle_types      3
demand           114
dtype: int64

In [13]:
stochastic_program.get_summary()

get_results_by_tuple_df finished in 0.39 seconds
get_results_by_region_df finished in 0.01 seconds
get_summary finished in 0.51 seconds


{'status': 'Optimal',
 'objective': 14138.135390625015,
 'expected_profit': 14052.060781250033,
 'eta': 14224.21,
 'beta': 0.5,
 'alpha': 0.75,
 'n_trips_avg': 5967.625,
 'n_unfilled_demand_avg': 3698.75,
 'demand_avg': 9666.375,
 'n_parking_avg': 2540.375,
 'n_relocations_avg': 2903.0}

# Benchmarks
We will now run different benchmarks on our model to evaluate its performance.
## Different Capacities/ Disabled Relocations/ Value Of Perfect Information
In our first benchmark we will solve the model with 3 different vehicle fleet sizes to see how the fleet size affects the performance of the model.  
For each fleet size we will also enable/disable relocations, so that we can see how relocations can improve profit and demand fulfillment. We will also enable/disable the non-anticipativty constraints to calculate the value of perfect information.

In [14]:
capacities = [
    {
        "kick_scooter": 10000,
        "bicycle": 4000,
        "car": 3000,
    },
    {
        "kick_scooter": 8000,
        "bicycle": 3000,
        "car": 2000,
    },
    {
        "kick_scooter": 6000,
        "bicycle": 2000,
        "car": 1000,
    },
]


In [15]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)
factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.00 seconds
_convert_demand finished in 0.04 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 0.04 seconds
_set_max_demand finished in 0.01 seconds
No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  -64271.778579658705
solve finished in 39.33 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  -62641.88248343602
solve finished in 17.69 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  -104458.20309417852
solve finished in 2.28 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  -104357.0079886999
solve finished in 2.04 seconds


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  -28270.7856219993
solve finished in 167.27 seconds


No parameters matchin

In [16]:
results_df = pd.DataFrame.from_dict(results)

In [17]:
results_df

,status,objective,expected_profit,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,kick_scooter,bicycle,car,relocations_disabled,non_anticipativity_disabled
0,Optimal,-64271.778580,-64271.778580,8865.875,800.500,9666.375,25110.250,8191.000,10000,4000,3000,False,False
1,Optimal,-62641.882483,-62641.882483,9301.625,364.750,9666.375,24698.375,8747.750,10000,4000,3000,False,True
2,Optimal,-104458.203094,-104458.203094,8171.125,1495.250,9666.375,25828.875,0.000,10000,4000,3000,True,False
3,Optimal,-104357.007989,-104357.007989,8177.125,1489.250,9666.375,25822.875,0.000,10000,4000,3000,True,True
4,Optimal,-28270.785622,-28270.785622,8585.375,1081.000,9666.375,17393.000,5638.000,8000,3000,2000,False,False
5,Optimal,-27090.620741,-27090.620741,8955.625,710.750,9666.375,17042.250,5711.625,8000,3000,2000,False,True
6,Optimal,-55696.431543,-55696.431543,7386.375,2280.000,9666.375,18613.625,0.000,8000,3000,2000,True,False
7,Optimal,-55616.336771,-55616.336771,7399.125,2267.250,9666.375,18600.000,0.000,8000,3000,2000,True,True
8,Optimal,4053.865424,4053.865424,8173.875,1492.500,9666.375,9818.750,3123.000,6000,2000,1000,False,False
9,Optimal,4583.889741,4583.889741,8375.875,1290.500,9666.375,9622.625,3553.250,6000,2000,1000,False,True


In [18]:
os.makedirs(os.path.dirname(PATH_RESULTS_SUMMARY), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SUMMARY)


## Single Modal Benchmark
The novelty of our model is the consideration of multiple vehicle types. With this benchmark we will examine whether this consideration actually improves the profit. To do that we will create three single modal models and run them subsequently. The first single modal model will only consider kick scooters. We will then extract the unfulfilled demand from the solution of the first model and use that as an additional input for the second single modal model, which considers bicycles. Therefore the second model will have the bicycle demand plus the unfulfilled demand from the kick scooters as input. We will then repeat the process for cars.  
We can then compare the sum of the profit of all three single modal models with the profit of the multi modal model. 

In [19]:
scenarios_copy = scenarios.copy().reset_index()

demand_per_type = {
    "car": scenarios_copy[scenarios_copy['vehicle_types'] == 'car']\
        .set_index(['scenarios','start_hex_ids','end_hex_ids','time' ,'vehicle_types']),
    "kick_scooter": scenarios_copy[scenarios_copy['vehicle_types'] == 'kick_scooter']\
        .set_index(['scenarios','start_hex_ids','end_hex_ids','time' ,'vehicle_types']),
    "bicycle": scenarios_copy[scenarios_copy['vehicle_types'] == 'bicycle']\
        .set_index(['scenarios','start_hex_ids','end_hex_ids','time' ,'vehicle_types']),
}

In [20]:
results = []
# last demand in first iteration is 0
last_demand = pd.DataFrame(index=demand_per_type['car'].index)
last_demand['demand'] = 0
for vehicle_types in [["kick_scooter"], ["bicycle"], ["car"]]:
    current_vehicle_type = vehicle_types[0]
    print("previous demand", demand_per_type[current_vehicle_type]['demand'].values.sum(), "\n")
    print("demand for current vehicle", last_demand['demand'].values.sum(), "\n")
    current_demand = demand_per_type[current_vehicle_type] \
                    .reset_index('vehicle_types') \
                    .drop("vehicle_types", axis=1) \
                    + last_demand.reset_index('vehicle_types').drop("vehicle_types", axis=1)
    current_demand['vehicle_types'] = current_vehicle_type
    current_demand = current_demand.set_index('vehicle_types', append=True)
    print("total current demand", current_demand['demand'].values.sum(), "\n")
    current_fleet_capacity = {
        current_vehicle_type: real_fleet_size[current_vehicle_type]
    }

    current_demand.index = current_demand.index.set_levels(
        current_demand.index.get_level_values("vehicle_types").map(
            lambda x: current_vehicle_type
        ),
        verify_integrity=False,
        level="vehicle_types",
    )

    factory = StochasticProgramFactory(
        current_demand,
        distances,
        probabilities,
        node_df,
        vehicle_types,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    stochastic_program.create_model()
    stochastic_program.solve()
    # we transform the unfulfilled demand of the current lp into the demand for the next lp
    last_demand = stochastic_program.get_unfulfilled_demand().rename(
        columns={"accumulated_unfulfilled_demand": "demand"}
    )
    
    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": str(vehicle_types)}
    )


previous demand 48838 

demand for current vehicle 0 

total current demand 48838 

No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  8070.489229109914
solve finished in 0.90 seconds
previous demand 25997 

demand for current vehicle 19899 

total current demand 45896 

No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  4787.744994036929
solve finished in 0.97 seconds
previous demand 2496 

demand for current vehicle 34131 

total current demand 36627 

No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  -7844.7254961705
solve finished in 0.90 seconds


In [21]:
factory = StochasticProgramFactory(
    scenarios,
    distances,
    probabilities,
    node_df,
    ALL_VEHICLE_TYPES,
    include_methods=[None],
)
factory.set_initial_allocation(real_fleet_size)
stochastic_program = factory.create_stochastic_program()
stochastic_program.include_methods = ["solve"]
stochastic_program.create_model()
stochastic_program.solve()

results.append(
    {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
)


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  14222.36351442462
solve finished in 11.50 seconds


In [22]:
results = pd.DataFrame.from_dict(results)

In [23]:
compare = results.iloc[[3]]
compare.append(results.iloc[range(3)].sum(), ignore_index=True)

,status,objective,expected_profit,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,vehicle_types
0,Optimal,14222.363514,14222.363514,5972.625,3693.750,9666.375,2535.125,2786.0,"[kick_scooter, bicycle, car]"
1,OptimalOptimalOptimal,5013.508727,5013.508727,6018.000,10402.125,16420.125,2490.000,2887.0,['kick_scooter']['bicycle']['car']


In [24]:
os.makedirs(os.path.dirname(PATH_RESULTS_SINGLE_MODAL_BENCHMARK), exist_ok=True)
compare.to_pickle(PATH_RESULTS_SINGLE_MODAL_BENCHMARK)

## Value Of The Stochastic Solution

In [25]:
demand = scenarios.copy()
demand = (
    demand.unstack("scenarios")["demand"]
    .sum(axis=1)
    .to_frame()
    .rename(columns={0: "demand"})
)
demand["scenarios"] = 0
demand = demand.set_index("scenarios", append=True).reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)
demand['demand'] = demand['demand'] / N_REDUCED_SCNEARIOS
demand['floored'] = demand['demand'].apply(np.floor)
demand['ceiled'] = demand['demand'].apply(np.ceil)


In [26]:
results = []
for rounding_mode in ["floored", "ceiled"]:
    factory = StochasticProgramFactory(
        demand[[rounding_mode]].rename(columns={rounding_mode: "demand"}),
        distances,
        probabilities,
        node_df,
        ALL_VEHICLE_TYPES,
        include_methods=[None],
    )
    factory.set_initial_allocation(real_fleet_size)
    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ["solve"]
    
    # assign all weight to first scenario
    stochastic_program.weighting = {0: 1}

    # discard non-anticipativity constraints
    stochastic_program.non_anticipativity_disabled = True

    stochastic_program.create_model()
    stochastic_program.solve()

    results.append(
        {**stochastic_program.get_summary(), "vehicle_types": ALL_VEHICLE_TYPES}
    )


No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  488.69696032763284
solve finished in 0.35 seconds
No parameters matching '_test' found
Status: Optimal
Optimal Value of Objective Function:  731.0341389736636
solve finished in 0.33 seconds


In [27]:
results = pd.DataFrame.from_dict(results)
results

,status,objective,expected_profit,n_trips_avg,n_unfilled_demand_avg,demand_avg,n_parking_avg,n_relocations_avg,vehicle_types
0,Optimal,488.696960,488.696960,5972.0,3451.0,9423.0,2536.0,2731.0,"[kick_scooter, bicycle, car]"
1,Optimal,731.034139,731.034139,5972.0,4138.0,10110.0,2536.0,2577.0,"[kick_scooter, bicycle, car]"


In [28]:
results_df.to_pickle(PATH_RESULTS_VALUE_STOCHASTIC)